In [ ]:
##### Import important modules #####

from os.path import join as opj
from nipype.interfaces.freesurfer import (BBRegister, ApplyVolTransform,
                                          Binarize, MRIConvert, FSCommand)
from nipype.interfaces.fsl import BET, BinaryMaths, ImageMaths
from nipype.interfaces.spm import (Realign, Smooth)
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.algorithms.misc import Gunzip
from nipype.pipeline.engine import Workflow, Node, MapNode


In [ ]:
# specify path to SPM-standalone
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/home/lmn/spm12/')

# set path to freesurfer directory
from nipype.interfaces.freesurfer import FSCommand
fs_dir = '/media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/mindboggle/freesurfer_subjects/' 
FSCommand.set_default_subjects_dir(fs_dir) 

In [ ]:
# set paths and define parameters
experiment_dir = '/media/lmn/86A406A0A406933B2/TNAC_BIDS/'
output_dir = 'derivatives/preprocessing/output_preproc'
working_dir = 'derivatives/preprocessing/workingdir_preproc'

# list of subjects (for practice only one)
subject_list = ['sub-15', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-21', 'sub-22', 'sub-23', 'sub-24']

# Smoothing widths to apply
fwhm = 3

# TR of functional images (from dataset)
TR = 1.45


In [ ]:
##### Create & specify nodes to be used and connected during the preprocessing pipeline #####

# Realign - correct for motion
realign = Node(Realign(register_to_mean=True),
               name="realign")

# Artifact Detection - determine which of the images in the functional series
#   are outliers. This is based on deviation in intensity or movement.
art = Node(ArtifactDetect(norm_threshold=1,
                          zintensity_threshold=3,
                          mask_type='file',
                          parameter_source='SPM',
                          use_differences=[True, False]),
           name="art")

meanfuncmask = Node(interface=BET(mask=True,
                                  no_output=False,
                                  frac=0.55,
                                  robust=True,
                                  output_type='NIFTI',
                                  out_file='meanfunc'),
                       name='meanfuncmask')

# Gunzip - unzip functional
gunzip = MapNode(Gunzip(), name="gunzip", iterfield=['in_file'])

# Smooth - to smooth the images with a given kernel
smooth = Node(Smooth(fwhm=fwhm),
              name="smooth")

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

# Binarize Cortex node - creates a binary map of cortical voxel
binarizeCortical = Node(Binarize(out_type='nii.gz',
                                    match = [3,42],
                                    binary_file='binarized_cortical.nii.gz'),
                        name='binarizeCortical')

# BBRegister - coregister a volume to the Freesurfer anatomical
bbregister = Node(BBRegister(init='spm',
                             contrast_type='t2',
                             out_fsl_file=True),
                  name='bbregister')

# Dilate node - dilates the binary brain mask
dilate_cortical_mask = Node(Binarize(dilate=1,
                          min=0.5),
              name='dilate_cortical_mask')

# MRIConvert - to unzip output files
mriconvert_cortical_mask = Node(MRIConvert(out_type='nii'),
                     name='mriconvert_cortical_mask')


# Binarize Dialte node - binarizes dilated mask again after transformation
binarizeDilatedCorticalMask = Node(Binarize(min=0.1, binary_file='dilated_cortical_mask.nii'),
                           name='binarizeDilatedCorticalMask')

applyVolTrans_cortical_mask = Node(ApplyVolTransform(reg_header=True,
                                  interp='nearest'),
                name='applyVolTrans_cortical_mask')


aparc_robust_BET_mask = MapNode(interface=ImageMaths(suffix='_ribbon_robust_BET',
                                               op_string='-mas', output_type='NIFTI', out_file='aparc_robust_BET.nii'),
                      iterfield=['in_file'],
                      name='aparc_robust_BET_mask')


# Gets path to the aparc+aseg file
def get_aparc_aseg(files):
    for name in files:
        if 'aparc+aseg' in name:
            return name

# Gets path to the ribbon file (both hemispheres)
def get_ribbon(files):
    for name in files:
        if 'ribbon.mgz' in name:
            return name

def get_maskmask(file):
    return file[0]

# plot realign parameters
def plot_realign_parameters(subject_id, realigned_files):

    # Import necessary modules
    from os.path import join as opj 
    import numpy
    import matplotlib.pyplot as plt
    import glob

    experiment_dir = '/media/lmn/86A406A0A406933B2/TNAC_BIDS/'
    working_dir = 'derivatives/preprocessing/workingdir_preproc'
    # Load the estimated parameters
    list_realign_parameter_files=glob.glob(opj(experiment_dir, working_dir, "preproc",  "_subject_id_"+subject_id, "realign", "*.txt"))
    
        
    plot_realign_parameters=[]
    for realign_file in list_realign_parameter_files:
    
        movement=numpy.loadtxt(realign_file)
    
        plt.figure(figsize=(10,8))
        
        plt.axhline(0, color='red')
        
        # Create the plots with matplotlib
        plt.subplot(211)
        plt.title('translation')
        plt.ylabel('in mm')
        plt.plot(movement[:,0], label='x')
        plt.plot(movement[:,1], label='y')
        plt.plot(movement[:,2], label='z')
        
        plt.legend(loc='best', fancybox=True)
        
        plt.axhline(0, color='red', alpha=0.5)
        
        plt.subplot(212)
        plt.title('rotation')
        plt.ylabel('in degrees')
        plt.plot(movement[:,3], label='pitch')
        plt.plot(movement[:,4], label='roll')
        plt.plot(movement[:,5], label='yaw')
        
        plt.legend(loc='best', fancybox=True)
        
        plt.axhline(0, color='red', alpha=0.5)

        plot_realign_parameters=plt.tight_layout()
        
        plt.savefig(opj(experiment_dir, working_dir, "preproc", "_subject_id_"+subject_id, "realign", realign_file+".png"), bbox_inches='tight')
        
    return plot_realign_parameters
    
# plot realing parameters - plot realignment parameters and save it later on
plot_realign_parameters = Node(Function(input_names=['subject_id', 'realigned_files'],
                               output_names=['plot_realign_parameters'],
                               function=plot_realign_parameters),
                      name='plot_realign_parameters')


In [ ]:
# Create a preprocessing workflow
preproc = Workflow(name='preproc')
preproc.base_dir = opj(experiment_dir, working_dir)

# Connect all components of the preprocessing workflow - aparc / ribbon mask 
preproc.connect([(gunzip, realign, [('out_file', 'in_files')]), # send unzipped files to realignment node
                 (realign, art, [('realigned_files', 'realigned_files')]), # send realigned files to artifact detection node
                 (realign, art, [('mean_image', 'mask_file'),
                                 ('realignment_parameters',
                                  'realignment_parameters')]),
    
                 (realign, meanfuncmask, [('mean_image', 'in_file')]), # create a skull stripped mask image from the mean functional
                 (realign, bbregister, [('mean_image', 'source_file')]), # coregister mean functional and anatomical
                 (binarizeCortical, mriconvert_cortical_mask, [('binary_file', 'in_file')]), # convert ribbon file (cortical mask) to nifti
                 (mriconvert_cortical_mask, applyVolTrans_cortical_mask,[('out_file','source_file')]), # transform ribbon file (cortical mask) to functional space
                 (realign, applyVolTrans_cortical_mask, [('mean_image', 'target_file')]),
                
                 (applyVolTrans_cortical_mask, dilate_cortical_mask, [('transformed_file', 'in_file')]), # dilate transformed ribbon file (cortical mask)
                 
                 (dilate_cortical_mask, binarizeDilatedCorticalMask, [('binary_file', 'in_file')]), # binarize dilated and transformed ribbon file (cortical mask)
                 
                 (meanfuncmask, aparc_robust_BET_mask, [('mask_file', 'in_file')]), # create a combined mask from ribbon file (cortical mask) and robust_BET mask
                 (binarizeDilatedCorticalMask, aparc_robust_BET_mask, [('binary_file', 'in_file2')]),
                                  
                 (realign, smooth, [('realigned_files', 'in_files')]), # send realigned files to smooth node
                 (realign, plot_realign_parameters, [('realigned_files', 'realigned_files')]), # send realign files / parameters to plot realignment parameters node
                 ])      
       

In [ ]:
##### Specify input and output stream #####

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]


# SelectFiles - to grab the data (alternativ to DataGrabber)
templates = {'func': '{subject_id}/func/*.nii.gz',
             'aparc_aseg': 'derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/aparc+aseg.mgz',
             'brainmask': 'derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/brainmask.mgz',
             'ribbon': 'derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/ribbon.mgz'}
selectfiles = Node(SelectFiles(templates,
                               base_directory=experiment_dir),
                   name="selectfiles")
       

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_subject_id_', '')]

datasink.inputs.substitutions = substitutions

In [ ]:
##### establish input and output streams by connecting Infosource, SelectFiles and DataSink to the main workflow #####
preproc.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                  (infosource, bbregister, [('subject_id',
                                          'subject_id')]),
                  (infosource, fssource, [('subject_id',
                                          'subject_id')]),
                  (selectfiles, gunzip, [('func', 'in_file')]),
                  (selectfiles, binarizeCortical, [('ribbon', 'in_file')]),
                  (realign, datasink, [('mean_image',
                                        'realign.@mean'),
                                       ('realignment_parameters',
                                        'realign.@parameters')]),
                  (art, datasink,     [('outlier_files',
                                        'art.@outliers'),
                                       ('plot_files',
                                        'art.@plot')]),
                  (binarizeDilatedCorticalMask, datasink,   [('binary_file',
                                        'masks.@binarized_dilated_cortical_mask')]),
                  (meanfuncmask, datasink, [('mask_file',
                                        'masks.@mean_func_mask')]),
                  (bbregister, datasink, [('out_reg_file',
                                        'bbregister.@out_reg_file'),
                                       ('out_fsl_file',
                                        'bbregister.@out_fsl_file'),
                                       ('registered_file',
                                        'bbregister.@registered_file')]),
                  (aparc_robust_BET_mask, datasink, [('out_file', 
                                                      'masks.@aparc_robust_BET_mask')]),
                  (smooth, datasink, [('smoothed_files', 'smooth.@smoothed_files')]),
                  (infosource, plot_realign_parameters, [('subject_id',
                                                  'subject_id')]),		
                  ])


In [ ]:

#### visualize the pipeline ####

# Create a colored mvpaflow output graph
preproc.write_graph(graph2use='colored',format='png', simple_form=True)

# Create a detailed mvpaflow output graph
preproc.write_graph(graph2use='flat',format='png', simple_form=True)

#### run the workflow using multiple cores ####

preproc.run('MultiProc', plugin_args={'n_procs':4})


In [ ]:
# Visualize the simple graph
from IPython.display import Image
Image(filename='/media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/preprocessing/workingdir_preproc/preproc/graph.png')

In [ ]:
Image(filename='/media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/preprocessing/workingdir_preproc/preproc/graph_detailed.png')

In [ ]:
!tree /media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/preprocessing/output_preproc/